In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline


In [13]:
text2text_generator = pipeline(task="text2text-generation")
text2text_generator("What is the climate of Canada like?")

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'generated_text': '         '}]

In [10]:
text2text_generator.model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [203]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots


def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [247]:
points = 1500

loudness = 25
validate_loudness = lambda x: x if x else 1

valence, energy, danceability = 0.5, 0.0, 0.3
acousticness, instrumentalness, speechiness = 0.5, 0.0, 0.3


theta = np.linspace(0, 48*np.pi, points)
plane = np.zeros(points)

valence_rho = loudness * np.cos(5 * theta * valence) + 1
energy_rho = loudness * np.cos(5 * theta * energy) + 1
danceability_rho = loudness * np.cos(5 * theta * danceability) + 1

acoustic_rho = validate_loudness(loudness) * (np.sin(2 * theta * speechiness) + np.cos(2 * theta * acousticness))
instrumental_rho = validate_loudness(loudness) * (np.sin(2 * theta * speechiness) + np.cos(2 * theta * instrumentalness))
speech_rho = validate_loudness(loudness) * (np.sin(2 * theta * speechiness) + np.cos(2 * theta * speechiness))

In [248]:
x_valence, y_valence = pol2cart(valence_rho, theta)
x_energy, y_energy = pol2cart(energy_rho, theta)
x_danceability, y_danceability = pol2cart(danceability_rho, theta)

x_acoustic, y_acoustic = pol2cart(acoustic_rho, theta)
x_instrumental, y_instrumental = pol2cart(instrumental_rho, theta)
x_speech, y_speech = pol2cart(speech_rho, theta)

In [319]:
fig = make_subplots(
    rows=1, 
    cols=2, 
    specs=[[{"type":"scatter3d"}, {"type":"scatter3d"}]],
    horizontal_spacing=.01,
    subplot_titles=["Vibes", "Musicality"]
)

vibe_df = pd.DataFrame(
    {
        "Attribute": (["Valence"] * points) + (["Energy"] * points) + (["Danceability"] * points),
        "X": np.concatenate((x_valence, plane, y_danceability)),
        "Y": np.concatenate((y_valence, x_energy, plane)),
        "Z": np.concatenate((plane, y_energy, x_danceability))
    }
)
# fig = px.line_3d(data_frame=vibe_df, x="X",y="Y", z="Z", color="Attribute", title="Vibes")
# fig.update_layout(
#     scene = dict(
#         bgcolor = "rgba(0,0,0,0)",
#     ),
#     scene_aspectmode='cube'
# )
# fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
# fig.show()
t1 = px.line_3d(data_frame=vibe_df, x="X",y="Y", z="Z", color="Attribute", width=1600, height=800)
t1.update_traces(legendgroup=1, legendgrouptitle=dict(text="Vibes"), legend="legend")
fig.add_traces(
    t1.data,
    rows=1,
    cols=1
)


musicality_df = pd.DataFrame(
    {
        "Attribute": (["Acousticness"] * points) + (["Instrumentalness"] * points) + (["Speechinees"] * points),
        "X": np.concatenate((x_acoustic, plane, y_speech)),
        "Y": np.concatenate((y_acoustic, x_instrumental, plane)),
        "Z": np.concatenate((plane, y_instrumental, x_speech))
    }
)
# fig = px.line_3d(data_frame=musicality_df, x="X",y="Y", z="Z", color="Attribute")
# fig.update_layout(
#    scene = dict(
#        bgcolor = "rgba(0,0,0,0)",
#    ),
#    scene_aspectmode='cube'
# )
# fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
# fig.show()
t2 = px.line_3d(data_frame=musicality_df, x="X",y="Y", z="Z", color="Attribute", width=1600, height=800)
t2.update_traces(legendgroup=2, legendgrouptitle=dict(text="Musicality"), legend="legend2")
fig.add_traces(
    t2.data,
    rows=1,
    cols=2
)

fig.update_layout(
    scene = dict(
        bgcolor = "rgba(0,0,0,0)",
    ),
    scene_aspectmode='cube',
    margin=dict(l=20, r=20, t=10, b=0),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=0.1,
        xanchor="center",
        x=0.5,
        tracegroupgap=10
    ),
    
)
fig.layout.annotations[0].update(y=0.85)
fig.layout.annotations[1].update(y=0.85)

fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
# fig.update_traces(legendgroup=2)

fig.show()